### Using the experimental FL Scatter-Gather DSL Node to submit the HELLOWORLD example 

### Requirements

Create AML resources by following this tutorial: 
https://github.com/Azure-Samples/azure-ml-federated-learning/blob/main/docs/quickstart.md

##### Import libraries

In [2]:
import os
import argparse
import random
import string
import datetime
import webbrowser
import time
import json
import sys

# Azure ML sdk v2 imports
import azure
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

# to handle yaml config easily
from omegaconf import OmegaConf

#### Load components

In [3]:
YAML_CONFIG = OmegaConf.load("config.yaml")
# path to the components
COMPONENTS_FOLDER = os.path.join(
    "./", "..", "..", "components", "HELLOWORLD"
)

# path to the shared components
SHARED_COMPONENTS_FOLDER = os.path.join(
    "./", "..", "..", "components", "utils"
)


preprocessing_component = load_component(
    source=os.path.join(COMPONENTS_FOLDER, "preprocessing", "spec.yaml")
)

training_component = load_component(
    source=os.path.join(COMPONENTS_FOLDER, "traininsilo", "spec.yaml")
)

aggregate_component = load_component(
    source=os.path.join(SHARED_COMPONENTS_FOLDER, "aggregatemodelweights", "spec.yaml")
)

#### Create ML client

In [4]:
# USER TODO: fill in these values if you wish to produce a client handle without going through online authentication.
subscription_id = '48bbc269-ce89-4f6f-9a12-c6f91fcb772d'
resource_group = 'amitgarg-fldev-rg'
workspace = 'aml-flhe'

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

if subscription_id is not None and resource_group is not None and workspace is not None:
    ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
    print("Client created from user-provided values.")
else:
    print("Client Unchanged.")

Client created from user-provided values.


#### Create Federated Learning Silos

In [5]:
from azure.ai.ml import Input
from azure.ai.ml.entities._assets.federated_learning_silo import FederatedLearningSilo

silo_list = [
    FederatedLearningSilo(
        compute=silo_config["compute"],
        datastore=silo_config["datastore"],
        inputs= {
            "silo_name": dict(silo_config["inputs"])["name"],
            "raw_train_data": Input(**dict(silo_config["inputs"])["raw_training_data"]),
            "raw_test_data": Input(**dict(silo_config["inputs"])["raw_testing_data"]),
        },
    )
    for silo_config in YAML_CONFIG.strategy.horizontal
]
print("Silo list created")

Silo list created


#### Create Argument Maps

In [6]:
silo_to_aggregation_argument_map = {"model" : "from_silo_input"}
aggregation_to_silo_argument_map = {"aggregated_output" : "checkpoint"}

#### Create kwarg input maps

In [7]:
silo_kwargs = dict(YAML_CONFIG.inputs)
agg_kwargs = {}

#### Create Silo subgraph

In [8]:
@pipeline(
    name="Silo Federated Learning Subgraph",
    description="It includes all steps that needs to be executing in silo",
)
def silo_scatter_subgraph(
    # user defined inputs
    raw_train_data: Input,
    raw_test_data: Input,
    checkpoint: Input(optional=True),
    silo_name: str,
    # user defined training arguments
    lr: float = 0.01,
    epochs: int = 3,
    batch_size: int = 64,
    dp: bool = False,
    dp_target_epsilon: float = 50.0,
    dp_target_delta: float = 1e-5,
    dp_max_grad_norm: float = 1.0,
) -> dict:
    """Create scatter/silo subgraph.

    Args:
        raw_train_data (Input): raw train data
        raw_test_data (Input): raw test data
        checkpoint (Input): if not None, the checkpoint obtained from previous iteration
        scatter_compute1 (str): Silo compute1 name
        scatter_compute2 (str): Silo compute2 name
        iteration_num (int): Iteration number
        lr (float, optional): Learning rate. Defaults to 0.01.
        epochs (int, optional): Number of epochs. Defaults to 3.
        batch_size (int, optional): Batch size. Defaults to 64.
        dp (bool, optional): Differential Privacy
        dp_target_epsilon (float, optional): DP target epsilon
        dp_target_delta (float, optional): DP target delta
        dp_max_grad_norm (float, optional): DP max gradient norm
        num_of_iterations (int, optional): Total number of iterations

    Returns:
        Dict[str, Outputs]: a map of the outputs
    """
    # we're using our own preprocessing component
    silo_pre_processing_step = preprocessing_component(
        # this consumes whatever user defined inputs
        raw_training_data=raw_train_data,
        raw_testing_data=raw_test_data,
        # here we're using the name of the silo compute as a metrics prefix
        metrics_prefix=silo_name,
    )

    # # Assigning the silo's first compute to the preprocessing component
    # silo_pre_processing_step.compute = silo_compute1

    # we're using our own training component
    silo_training_step = training_component(
        # with the train_data from the pre_processing step
        train_data=silo_pre_processing_step.outputs.processed_train_data,
        # with the test_data from the pre_processing step
        test_data=silo_pre_processing_step.outputs.processed_test_data,
        # and the checkpoint from previous iteration (or None if iteration == 1)
        checkpoint=checkpoint,
        # Learning rate for local training
        lr=lr,
        # Number of epochs
        epochs=epochs,
        # Dataloader batch size
        batch_size=batch_size,
        # Differential Privacy
        dp=dp,
        # DP target epsilon
        dp_target_epsilon=dp_target_epsilon,
        # DP target delta
        dp_target_delta=dp_target_delta,
        # DP max gradient norm
        dp_max_grad_norm=dp_max_grad_norm,
        # Silo name/identifier
        metrics_prefix=silo_name,
    )

    # # Assigning the silo's second compute to the training component
    # silo_training_step.compute = silo_compute1

    # IMPORTANT: we will assume that any output provided here can be exfiltrated into the orchestrator/gather
    return {
        # NOTE: the key you use is custom
        # a map function scatter_to_gather_map needs to be provided
        # to map the name here to the expected input from gather
        "model": silo_training_step.outputs.model
    }

### Putting it all together - Creating a Pipeline with an FLCG Node

In [10]:
from azure.ai.ml.dsl import pipeline
import azure.ai.ml.dsl._fl_scatter_gather_node as fl
from azure.ai.ml.entities._credentials import UserIdentityConfiguration


iterations = 2 # arbitrary value set for example
@pipeline(default_compute=YAML_CONFIG.orchestrator.compute)
def fl_pipeline():
    fl_node = fl.fl_scatter_gather(
        silo_configs=silo_list,
        silo_component=silo_scatter_subgraph,
        aggregation_component=aggregate_component,
        aggregation_compute=YAML_CONFIG.orchestrator.compute,
        aggregation_datastore=YAML_CONFIG.orchestrator.datastore,
        shared_silo_kwargs=silo_kwargs,
        aggregation_kwargs=agg_kwargs,
        silo_to_aggregation_argument_map=silo_to_aggregation_argument_map,
        aggregation_to_silo_argument_map=aggregation_to_silo_argument_map,
        max_iterations=iterations, 
    )
    # NOTE: If you wish to examine the client-side subgraph, uncomment the following line (warning: large output)
    # print(fl_node.subgraph)

fl_pipeline_job = fl_pipeline()
# fl_pipeline_job.identity = UserIdentityConfiguration()
fl_pipeline_job = ml_client.jobs.create_or_update(fl_pipeline_job, experiment_name="example_fl_pipeline")
fl_pipeline_job

Operation failed. Retrying in 1.236599640902055 seconds.
Uploading _builders (0.49 MBs): 100%|##########| 485444/485444 [00:00<00:00, 905478.10it/s] 




Experiment,Name,Type,Status,Details Page
example_fl_pipeline,mango_spinach_qrtmxn1h3f,pipeline,Preparing,Link to Azure Machine Learning studio
